In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from model_super_resolution import Net
from super_resolution_data_loader import *

torch.manual_seed(1)
device = torch.device("cuda")

In [2]:
# SubPixelCNN parameters

batch_size = 32
epochs = 300
lr = 0.01
threads = 4
upscale_factor = 4

In [3]:
img_path_low = 'E:\\Datasets\\3DFaces\\300W-3D-ALL\\images-low-res'
img_path_ref = 'E:\\Datasets\\3DFaces\\300W-3D-ALL\\images-128'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [11]:
print('===> Building model')
model = Net(upscale_factor=upscale_factor).to(device)
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=lr)

===> Building model


In [12]:
def train(epoch):
    epoch_loss = 0
    for iteration, batch in enumerate(training_data_loader, 1):
        input_, target = batch[0].to(device), batch[1].to(device)
        
        optimizer.zero_grad()
        loss = criterion(model(input_), target)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        #print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))
    
    psnr_epoch = 10*log10(1/(epoch_loss / len(training_data_loader)))
    print("===> Epoch {} Complete: Avg. Loss: {:.4f} / PSNR: {:.4f}".format(epoch, epoch_loss / len(training_data_loader), psnr_epoch))

'''
def test():
    avg_psnr = 0
    with torch.no_grad():
        for batch in testing_data_loader:
            input, target = batch[0].to(device), batch[1].to(device)

            prediction = model(input)
            mse = criterion(prediction, target)
            psnr = 10 * log10(1 / mse.item())
            avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))
'''

def checkpoint(epoch):
    model_out_path = "trained_models/super_res_model_epoch_{}.pth".format(epoch)
    torch.save(model, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

In [13]:
for epoch in range(1, epochs + 1):
    train(epoch)
    #test()
    if epoch % 30 == 0:
        checkpoint(epoch)

===> Epoch 1 Complete: Avg. Loss: 0.0380 / PSNR: 14.2031
===> Epoch 2 Complete: Avg. Loss: 0.0054 / PSNR: 22.7129
===> Epoch 3 Complete: Avg. Loss: 0.0041 / PSNR: 23.8541
===> Epoch 4 Complete: Avg. Loss: 0.0038 / PSNR: 24.1839
===> Epoch 5 Complete: Avg. Loss: 0.0037 / PSNR: 24.2990
===> Epoch 6 Complete: Avg. Loss: 0.0037 / PSNR: 24.2603
===> Epoch 7 Complete: Avg. Loss: 0.0034 / PSNR: 24.7126
===> Epoch 8 Complete: Avg. Loss: 0.0038 / PSNR: 24.2561
===> Epoch 9 Complete: Avg. Loss: 0.0033 / PSNR: 24.8488
===> Epoch 10 Complete: Avg. Loss: 0.0044 / PSNR: 23.5665
===> Epoch 11 Complete: Avg. Loss: 0.0037 / PSNR: 24.3078
===> Epoch 12 Complete: Avg. Loss: 0.0032 / PSNR: 24.9487
===> Epoch 13 Complete: Avg. Loss: 0.0032 / PSNR: 24.9990
===> Epoch 14 Complete: Avg. Loss: 0.0032 / PSNR: 24.9193
===> Epoch 15 Complete: Avg. Loss: 0.0031 / PSNR: 25.0619
===> Epoch 16 Complete: Avg. Loss: 0.0040 / PSNR: 23.9698
===> Epoch 17 Complete: Avg. Loss: 0.0031 / PSNR: 25.0804
===> Epoch 18 Complete: